In [31]:
import json

with open('documents-with-id.json', 'r') as f_in:
    documents = json.load(f_in)

In [32]:
from sentence_transformers import SentenceTransformer

In [33]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'

In [34]:
model = SentenceTransformer(model_name)

In [35]:
v = model.encode('My name is Fabio')

In [36]:
v.dot(v)

1.0

In [37]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id" : {"type": "keyword"},
            "question_vector": {"type" : "dense_vector", "dims":384, "index" : True, "similarity" : "cosine"},
            "text_vector": {"type" : "dense_vector", "dims":384, "index" : True, "similarity" : "cosine"},
            "question_text_vector": {"type" : "dense_vector", "dims":384, "index" : True, "similarity" : "cosine"}
        }
    }
}

index_name = 'course-questions'

es_client.indices.delete(index = index_name, ignore_unavailable = True)
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [38]:
for doc in documents:
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)


In [39]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [40]:
query = 'I just discovered the course, can I still join?'

In [41]:
v_q = model.encode(query)

In [42]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [44]:
elastic_search_knn('question_vector', v_q, 'data-engineering-zoomcamp')

[{'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'id': 'da77a135'},
 {'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'id': '5cb59d48'},
 {'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp',
  'section': '

In [45]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [46]:
import pandas as pd
df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]

{'question': 'When is the course start date and time?',
 'course': 'data-engineering-zoomcamp',
 'document': '2f401745'}

In [47]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [49]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [52]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in ground_truth:
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [53]:
evaluate(ground_truth, question_vector_knn)

{'hit_rate': 0.7678921035457907, 'mrr': 0.6625697918932646}

In [54]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [56]:
evaluate(ground_truth, text_vector_knn)

{'hit_rate': 0.8220578638242332, 'mrr': 0.6988506997317101}

In [58]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [59]:
evaluate(ground_truth, question_text_vector_knn)

{'hit_rate': 0.9088536001740265, 'mrr': 0.8107497643390622}